In [6]:
from navec import Navec
from slovnet import NER
from ipymarkup import show_span_box_markup as show_span_markup, show_ascii_markup as show_markup, show_dep_ascii_markup as show_dep_markup
from slovnet import Morph, Syntax
from razdel import sentenize, tokenize

from yargy import Parser, rule, and_, not_, or_
from yargy.interpretation import fact
from yargy.predicates import gram, tag, custom, type, in_
from yargy.relations import gnc_relation
from yargy.pipelines import morph_pipeline

import pymorphy2
import nltk
import re

pymorphy = pymorphy2.MorphAnalyzer(lang='ru')

navec = Navec.load('./data/navec_news_v1_1B_250K_300d_100q.tar')

ner = NER.load('./data/slovnet_ner_news_v1.tar')
morph = Morph.load('./data/slovnet_morph_news_v1.tar', batch_size=4)
syntax = Syntax.load('./data/slovnet_syntax_news_v1.tar')

ner.navec(navec)
morph.navec(navec)
syntax.navec(navec)

print("load: ok")

load: ok


In [61]:
source_txt ="""Совладельцы сети магазинов фиксированных низких цен Fix Price Артем Хачатрян и Сергей Ломакин стали долларовыми миллиардерами после выхода компании на IPO (первичное публичное размещение акций). Об этом свидетельствуют подсчеты Forbes.
Журнал оценил состояние каждого из них в 3,4 миллиарда долларов. Bloomberg написал о 3,6 миллиарда.
Fix Price разместила свои бумаги на Лондонской бирже по цене 9,75 доллара за штуку. Ломакину и Хачатряну до размещения принадлежало по 354 067 500 акций (41,66%) компаний, сейчас — по 301 151 876 акций (35,43%). Таким образом, в ходе IPO каждый из них продал по 52,9 миллиона акций, и заработал по 516 миллионов долларов, подсчитал Forbes.
Ранее сеть Fix Price, бумаги которой с 10 марта 2021 года начнут торговаться на Лондонской и Московской биржах, сообщила, что привлечет в ходе IPO 2 миллиарда долларов.
«Интерфакс» писал, что это крупнейшее IPO российской компании с 2010 года, когда «Русал» в рамках первичного публичного размещения акций привлек 2,24 миллиарда долларов.
Первые магазины сети Fix Price были открыты в 2007 году. Сейчас у компании 4279 магазинов в России, Беларуси, Казахстане, Узбекистане, Кыргызстане, Грузии и Латвии. Вначале весь ассортимент в Fix Price продавался по 30 рублей за товар, сейчас товары там стоят не дороже 250 рублей.
Артем Хачатрян и Сергей Ломакин в 1998 году стали сооснователями сети дискаунтеров «Копейка» (сейчас эти магазины известны под брендом «Пятерочка»). В 2007 году они продали свою долю корпорации «Уралсиб» за 220 миллионов долларов. Также Хачатрян и Ломакин инвестировали в сети «Модис» и «ЦентрОбувь».
В мае 2020 года Forbes писал, что в России больше 100 долларовых миллиардеров.
"""

In [8]:
source_txt = """
На сайте экспертного центра НАТО «Атлантический совет» вышла нехарактерная для него статья.
В ней авторы рекомендуют странам военного альянса применять к России более гибкую внешнюю политику, основанную не на очередных санкциях за нарушения прав человека, а на взаимных стратегических интересах.
А демократизация России, говорится в статье, вообще может быть невыгодна США, ведь тот же Алексей Навальный — националист и поддерживал Кремль в крымском вопросе.
Статья вызвала громкий скандал в экспертных кругах и ответные открытые письма с обвинениями авторов в коррупции.
"""

In [9]:
source_txt = """
Компания «Яндекс» ведет переговоры о покупке небольшого банка «Акрополь», 
который принадлежит генеральному директору группы «Связной» Евгению 
Давыдовичу. Об этом сообщает The Bell со ссылкой на четыре источника 
на платежном и банковском рынке. 
Сделка, по данным издания, уже находится на одном из завершающих этапов. 
Договор купли-продажи может быть подписан в ближайшие несколько дней. Сумму
сделки The Bell не приводит. 
Активы «Акрополя» составляют 1,24 миллиарда рублей. В рейтинге крупнейших 
банков «Интерфакс-100» он занимает 323-ю строчку.
Евгений Давыдович отказался от комментариев. Представитель «Яндекса» 
сообщил The Bell, что компании «интересно получение банковской лицензии». 
«Мы рассматриваем все доступные варианты», — добавил он.
"""

In [10]:
source_txt = """
Режиссер фильма «Супернова» Гарри Макквин подверг критике действия 
прокатчика картины в России, вырезавшего сцену, где гей-пара пытается заняться 
сексом. Заявление режиссера приводит издание The Advocate. 
Мы, создатели фильма, самым решительным образом возражаем против цензуры
«Суперновы» в России. Вызывает глубокую обеспокоенность факт того, что 
фильм был смонтирован против нашей воли и без нашего разрешения.
Макквин подчеркнул, что команда создателей фильма понимает, что 
на дистрибьютора оказывалось давление, но «не потерпит цензуры такого рода». 
В самой компании World Pictures, занимающейся распространением фильма 
в России, от комментариев отказались. 
Фильм «Супернова» рассказывает о жизни гей-пары, в которой один 
из партнеров теряет память из-за деменции. Из российской версии картины был 
вырезан трехминутный эпизод, в котором мужчины пытаются заняться сексом. 
В 2019 году в российском прокате похожей цензуре был подвергнут фильм 
«Рокетмен», снятый по мотивам биографии певца Элтона Джона. В РФ фильм 
показывался без сцены секса двух мужчин и финальных титров, в которых 
говорилось, что сейчас певец живет в браке с мужчиной. Создатели фильма 
и сам Элтон Джон подвергли это решение критике.
"""

In [11]:
source_txt = """
Международный спортивный арбитраж (CAS) в Лозанне отклонил запрос 
Олимпийского комитета России, предлагавшего использовать песню «Катюша» 
вместо российского гимна на Олимпийских играх в Токио и Пекине.
«Комиссия CAS считает, что [понятие] „любой гимн, связанный с Россией“, 
распространяется на любую песню, связанную с Россией или со ссылками на нее, 
в том числе с „Катюшей“», — сказано в решении суда, которое цитирует Associated 
Press.
В декабре 2019 года Всемирное антидопинговое агентство отстранило Россию 
от Олимпийских и Паралимпийских игр, а также чемпионатов мира из-за 
манипуляций с допинг-пробами, которые содержались в базе данных 
Московской антидопинговой лаборатории.
Всемирное антидопинговое агентство планировало отстранить Россию 
от международных соревнований на четыре года, но в декабре 2020 года 
спортивный арбитраж решил сократить срок наказания до двух лет. Таким 
образом, российские спортсмены не смогут до 16 декабря 2022 года выступать 
под национальным флагом, а также использовать национальный гимн и название 
сборной России.
"""

In [12]:
source_txt = """
Форум проводит организация «Объединенные демократы», которая 
не является «нежелательной». По данным «Дождя», задержания мотивированы 
деятельностью «Открытой России», которая, по меньшей мере формально, 
не имеет отношения к проведению форума. Зарегистрированная в Великобритании 
организация с таким названием была еще в 2017 году признана 
в РФ «нежелательной», после чего началось преследование членов одноименного 
движения «Открытая Россия». В 2019 году оно объявило о ликвидации, и активисты
этой организации учредили другую структуру с аналогичным названием, 
но не получили регистрацию в Минюсте. «Обычно, раньше по крайней мере, 
приходили по статье о сотрудничестве с нежелательной организацией 
на мероприятия „Открытой России“. Каким образом полиция связала „Открытку“ 
и „Объединенных демократов“, я не знаю. Это абсолютно разные организации», — 
сказала координатор «Открытой России» Татьяна Усманова. 
"""

In [13]:
source_txt = """
Первые упоминания о невзаимозаменяемых токенах, они же NFT, появились в 2017 
году — тогда, например, проект Larva Labs запустил эксперимент под названием 
CryptoPunks. Он представлял собой коллекцию из 10 тысяч цифровых аватаров — 
по сути, графических изображений разных лиц, всякое из которых уникально, 
то есть найти две похожие картинки невозможно. При этом каждое изображение 
«привязали» к фрагменту компьютерного кода в блокчейн-платформе Ethereum — 
таким образом, у него был уникальный токен, в котором, в частности, хранилась 
информация о владельце изображения.
В итоге скачать эту картинку, как и любое другое изображение в интернете, мог кто 
угодно, однако ее единственным официальным владельцем являлся именно тот, чей
Ethereum-кошелек был указан в NFT к этой картинке. 
В чем смысл обладания изображением, которое любой человек может 
беспрепятственно скачать в интернете и, например, сделать своей аватаркой 
в соцсетях? Понять это можно, лишь рассмотрев эти аватары как произведения 
искусства и, в частности, проведя параллели между ними и «материальными» 
объектами вроде картин или скульптур. Репродукцию «Девочки с персиками» тоже 
может повесить у себя на кухне любой человек, но существует оригинал, который 
хранится в Третьяковской галерее, и у него имеется владелец. Чем популярнее 
становится картина (в том числе, чем больше ее репродукций «пойдет в народ»), 
тем выше может быть цена на оригинал.
"""

In [14]:
source_txt = """
Выгорание 🔥🚬

Сегодня общались с коллегой на тему выгорания. Если вам кажется, что вы ни разу не сталкивались с этим, то скорее всего вы просто не доходили до критических стадий.

Выгорание свойственно любой профессии, но особенно часто встречается в it сфере. Проблема серьезная и разбирать каждый отдельный случай должны специалисты, но представлять, что это и как не довести себя до критических стадий, нужно каждому. Как-то раз я смотрел очень полезную лекцию на эту тему, поделюсь тем, что запомнил и что применяю сам.

Вам когда-нибудь казалось, что вы не получаете удовольствие от работы? Что не хочется делать рабочие задачи и даже то, что раньше в работе вас радовало, кажется каким-то скучным и даже неприятным? А цели у вас просто нет и вы живёте в дне сурка, где нет ни единого просвета? Так проявляются начальные стадии выгорания. Если не обратить на это должного внимания, то дальше все только усугубится.

Сама проблема заключается в постепенном угасании «энергии», которой в конечном итоге хватает только на лежание на диване. На самом деле есть научное обоснование, связанное с выработкой нейромедиаторов - дофамина, серотонина и прочих. Лучше загуглить, но я расскажу вкратце. Дофамин отвечает за вашу мотивацию и самое главное - удовольствие от выполненной задачи. Получается такой «наркотик» - вы хотите что-то крутое сделать, чтобы испытать удовлетворение. Если уровни нейромедиаторов снижаются, человек впадает в депрессию.

Разделяют 4 стадии выгорания. На первой и второй выгорание совсем незаметно, его легко спутать с усталостью. Третья стадия уже выделяется и вы можете понять - да, я выгорел, мне не нравится моя деятельность. А на четвёртой не хочется делать абсолютно ничего, полная апатия и отвращение ко всему. Хорошая новость в том, что из первых трёх стадий можно выйти самостоятельно.

А как выйти из такого состояния, расскажем в следующем посте 😉
"""

In [55]:
source_txt = """
Близкий друг и соратник Эдварда — Амиран Сардаров. Тоже блогер. И вот этот Сардаров — делал интервью с Собяниным (дифирамбы пел), а также задавал вопросы на прямой линии с Путиным. Эта парочка Сардаров — Эдвард Бил, давно вызывает вопросы в блогерском сообществе. Им с рук сходит буквально все. Поэтому ответ на вопрос «а как так вышло?», напрашивается довольно очевидный. Коррупция и кумовство. Кумовство и коррупция.
"""

In [56]:
class Token:
    def __init__(self, token):
        self.token = token
        self.morph = None
        self.syntax = None
    
    def set_morph(self, morph):
        self.morph = morph
        
    def set_syntax(self, syntax):
        self.syntax = syntax
        
    def __repr__(self):
        return self.token

class Sentence:
    def __init__(self, sentence):
        self.sentence = sentence
        self.tokens = []
        self.clean_tokens = []
        self.syntax_tree = [] 

        self.tokenize()
        self.clean()
        
    @property
    def words(self):
        return list(map(str, self.tokens))

    def tokenize(self):
        self.tokens = [Token(_.text) for _ in tokenize(self.sentence)]
        
    def clean(self):
        clean_text = re.sub('[^а-яА-Яa-zA-Z]', ' ', self.sentence)
        clean_text = re.sub(r'\s+', ' ', clean_text)
        self.clean_tokens = [Token(_.text) for _ in tokenize(clean_text)]
        
    def morph(self):
        markup = morph(self.words)

        for i, token in enumerate(markup.tokens):
            self.tokens[i].set_morph(token)
            
    def syntax(self):
        markup = syntax(self.words)

        for i, token in enumerate(markup.tokens):
            self.tokens[i].set_syntax(token)

            source = int(token.head_id) - 1
            target = int(token.id) - 1

            if source > 0 and source != target:  # skip root, loops
                self.syntax_tree.append([source, target, token.rel])
                
    def show_syntax(self):
        if self.syntax_tree == []:
            self.syntax()

        show_dep_markup(self.words, self.syntax_tree)
        
    def __str__(self):
        return self.sentence
        
    def __repr__(self):
        s = ''
        for tok in self.tokens:
            s += '[{}] '.format(tok)
        return s


class Text:
    def __init__(self, text):
        self.text = text
        self.sentences = []
        
        self.sentenize()
        
    def sentenize(self):
        self.sentences = [Sentence(_.text) for _ in sentenize(self.text)]
        
    def morph_all_sentences(self):
        for sent in self.sentences:
            sent.morph()
            
    def syntax_all_sentences(self):
        for sent in self.sentences:
            sent.syntax()

    def __str__(self):
        return self.text
        
    def __repr__(self):
        return '\n\n Sentence: '.join([str(sent) for sent in self.sentences])

# Сегментация

In [57]:
text = Text(source_txt)

# Морфологический и синтаксический разбор 

In [58]:
text.morph_all_sentences()

In [59]:
text.syntax_all_sentences()

In [60]:
sentence = text.sentences[0]

In [61]:
sentence.show_syntax()

    ┌► Близкий  amod
  ┌─└─ друг     
  │ ┌► и        cc
┌─└►└─ соратник conj
│ └──► Эдварда  nmod
│   ┌► —        punct
└►┌─└─ Амиран   appos
│ └──► Сардаров flat:name
└────► .        punct


# Извлечение именованных сущностей

In [62]:
markup = ner(text.text)

In [63]:
show_span_markup(markup.text, markup.spans)

# Поиск словосочетаний

In [64]:
def show_matches(rule, *lines):
    parser = Parser(rule)
    for line in lines:
        matches = parser.findall(line)
        matches = sorted(matches, key=lambda _: _.span)
        spans = [_.span for _ in matches]
        show_markup(line, spans)
        if matches:
            facts = [_.fact for _ in matches]
            if len(facts) == 1:
                facts = facts[0]
            display(facts)

In [65]:
def exact_gram(tag):
    def check(v):
        parse = pymorphy.parse(v)[0]
        if not parse:
            return False

        return parse.tag.POS == tag
    
    return custom(check)

INT = type('INT')
NOUN = gram('NOUN')
ADJF = gram('ADJF')
ABBR = gram('Abbr')
VERB = gram('VERB')
PREP = gram('PREP')

EQ_NOUN = exact_gram('NOUN')

NUMBER = rule(
    INT.optional(),
    in_('.,').optional(),
    INT
)

LAST = and_(
    gram('Surn'),
    not_(gram('Abbr')),
)
FIRST = and_(
    gram('Name'),
    not_(gram('Abbr')),
)

gnc = gnc_relation()

In [66]:
NounPhrase = fact('NounPhrase', ['adj', 'noun'])

NumberPhrase = fact('NumberPhrase', ['number', 'noun'])

Fact = fact('Fact', ['value'])

Name = fact(
    'Name',
    ['first', 'last'],
)

NounPhraseRule = rule(
    ADJF.optional().repeatable().interpretation(
        NounPhrase.adj
    ).match(gnc),

    EQ_NOUN.repeatable().interpretation(
        NounPhrase.noun
    ).match(gnc)
).interpretation(NounPhrase).interpretation(Fact.value)

NumberPhraseRule = rule(
    NUMBER.interpretation(
        NumberPhrase.number
    ),
    EQ_NOUN.interpretation(
        NumberPhrase.noun
    )
).interpretation(NumberPhrase).interpretation(Fact.value)

NameRule = rule(
    FIRST.interpretation(
        Name.first
    ).match(gnc),
    LAST.interpretation(
        Name.last
    ).match(gnc)
).interpretation(
    Name
)

AllRules = rule(or_(
    NumberPhraseRule,
    NounPhraseRule
)).interpretation(Fact)

In [67]:
show_matches(AllRules, text.sentences[5].sentence)

Поэтому ответ на вопрос «а как так вышло?», напрашивается довольно 
        ─────    ──────                                            
очевидный.


[Fact(
     value=NounPhrase(
         adj=None,
         noun='ответ'
     )
 ),
 Fact(
     value=NounPhrase(
         adj=None,
         noun='вопрос'
     )
 )]

# Построение короткого предложения по дереву

In [68]:
def find_token_head(sentence, token):
    return next((t for t in sentence.tokens if t.syntax.id == token.syntax.head_id), None)

def find_token_children(sentence, token):
    return list(filter(lambda t: t.syntax.head_id == token.syntax.id, sentence.tokens))

def distance_from_root(sentence, token, was_here=[]):
    if token.syntax.rel in ['root', 'ccomp', 'xcomp', 'acl', 'acl:relcl', 'parataxis', 'advcl'] or not token or token in was_here:
        return 0
    
    append = 1
    
    if token.syntax.rel in ['conj', 'obj', '' 'cc']:
        append = 0
    
    return append + distance_from_root(sentence, find_token_head(sentence, token), was_here + [token])

def shorten(sentence, percent=0.5, show_syntax=True):
    filtered_tokens = []
    
    required_filter = lambda t: t.syntax.rel in ['nsubj', 'obj', 'root']
    
    nummod_filter = lambda t: t.syntax.rel.startswith('nummod')
    advmod_filter = lambda t: t.syntax.rel.startswith('advmod')
    case_filter = lambda t: t.syntax.rel == 'case'
    empty_modifier_filter = lambda t: t.syntax.rel in ['nmod', 'amod'] and len(find_token_children(sentence, t)) == 0
    discourse_filter = lambda t: t.syntax.rel in ['discourse']
    
    if show_syntax:
        sentence.show_syntax()

    root = next((t for t in sentence.tokens if t.syntax.rel == 'root'), None)
    
    if not root:
        return
    
    tokens_distance = [(t, distance_from_root(sentence, t)) for t in sentence.tokens]
    max_depth = max(t[1] for t in tokens_distance)
    max_allowed_token_depth = max_depth * percent

    was_changed = True
    
    # TODO тут наверно стоит все-таки проходиться от корней, чтобы корректно учитывать, какие токены включены, какие нет.
    # Иначе из-за порядка обхода теряется часть токенов (родитель добавляется позже ребенка и некоторые правила для ребенка не срабатывают)
    # Уже переделал на while, чтобы не переписывать логику обхода. Можно потом переписать более оптимально
    
    # TODO еще стоит разделять предложения на составные части (сложносочиненные/подчиненные),
    # чтобы учитывать макс. дистанцию от корня в конкретной ветке предложения

    i = -1
    while was_changed:
        i += 1
        was_changed = False

        def push_to_result(t):
            nonlocal was_changed
            if t.syntax.id not in map(lambda t: t.syntax.id, filtered_tokens):
                filtered_tokens.append(t)
                was_changed = True

        for t, d in tokens_distance:
            t_head = find_token_head(sentence, t)
            t_head_included = t_head and t_head.syntax.id in map(lambda t: t.syntax.id, filtered_tokens)

            # берем только top-N слов от корня
            # исключаем "висячие" модификаторы, они могут быть включены по другим правилам ниже
            if (d <= max_allowed_token_depth or required_filter(t)) and \
                not empty_modifier_filter(t) and \
                not discourse_filter(t):
                push_to_result(t)

            if t_head_included and t_head:
                # берем все числа, чтобы не потерять смысл
                if nummod_filter(t):
                    push_to_result(t)
                
                # на <- IPO, nmod -> case. Чтобы предложение оставалось читаемым, оставляем предлоги, чтобы не было висячих существительных
                if case_filter(t):
                    push_to_result(t)

                # выхода -> компании, obl -> any
                if t_head.syntax.rel == 'obl':
                    push_to_result(t)

                # не <- дороже, advmod -> advmod
                if t_head.syntax.rel == t.syntax.rel and t.syntax.rel in ['advmod', 'amod']:
                    push_to_result(t)

                # берем существительные модификаторы к объектам и субъектам предложения
                if t_head.syntax.rel in ['nsubj', 'obj'] and t.syntax.rel in ['nmod']:
                    push_to_result(t)
    
    print(i + 1, 'iterations')
    
    filtered_tokens.sort(key=lambda t: int(t.syntax.id))

    return list(map(lambda t: t.token, filtered_tokens))

In [69]:
total_src_len = 0
total_short_len = 0

for sent in text.sentences:
    short = shorten(sent, 0.5, True)
    
    if not short:
        continue
    
    src_len = len(sent.tokens)
    short_len = len(short)
    
    print(sent, src_len)
    print(' '.join(short), short_len)
    print('-----------\n\n')
    
    total_short_len += short_len
    total_src_len += src_len
    
print('src len', total_src_len, 'short', total_short_len, 'diff', total_src_len - total_short_len)
print('{:.0f}%'.format(total_short_len / total_src_len * 100))

    ┌► Близкий  amod
  ┌─└─ друг     
  │ ┌► и        cc
┌─└►└─ соратник conj
│ └──► Эдварда  nmod
│   ┌► —        punct
└►┌─└─ Амиран   appos
│ └──► Сардаров flat:name
└────► .        punct
  ┌► Тоже   advmod
┌─└─ блогер 
└──► .      punct
2 iterations
Тоже блогер. 3
блогер 1
-----------


    ┌──────► И         cc
    │ ┌────► вот       advmod
    │ │   ┌► этот      det
    │ │ ┌►└─ Сардаров  nsubj
    │ │ │ ┌► —         punct
┌─┌─└─└─└─└─ делал     
│ │   ┌─└──► интервью  obj
│ │   │   ┌► с         case
│ │ ┌─└──►└─ Собяниным nmod
│ │ │   ┌──► (         punct
│ │ │   │ ┌► дифирамбы nsubj
│ │ └──►└─└─ пел       parataxis
│ │     └──► )         punct
│ │   ┌────► ,         punct
│ │   │ ┌►┌─ а         cc
│ │   │ │ └► также     fixed
│ └──►└─└─┌─ задавал   conj
│     ┌───└► вопросы   obj
│     │ ┌──► на        case
│     │ │ ┌► прямой    amod
│     └►└─└─ линии     nmod
│     │   ┌► с         case
│     └──►└─ Путиным   nmod
└──────────► .         punct
3 iterations
И вот этот Сардаров

# Text summarization

## BERT

In [70]:
from summarizer import Summarizer

In [71]:
model = Summarizer()

In [72]:
short_text = model(source_txt, num_sentences=3)

In [73]:
print(source_txt)

print('-----')

print(short_text)


Близкий друг и соратник Эдварда — Амиран Сардаров. Тоже блогер. И вот этот Сардаров — делал интервью с Собяниным (дифирамбы пел), а также задавал вопросы на прямой линии с Путиным. Эта парочка Сардаров — Эдвард Бил, давно вызывает вопросы в блогерском сообществе. Им с рук сходит буквально все. Поэтому ответ на вопрос «а как так вышло?», напрашивается довольно очевидный. Коррупция и кумовство. Кумовство и коррупция.

-----
Близкий друг и соратник Эдварда — Амиран Сардаров. И вот этот Сардаров — делал интервью с Собяниным (дифирамбы пел), а также задавал вопросы на прямой линии с Путиным. Поэтому ответ на вопрос «а как так вышло?»,


## Частотная суммаризация

In [74]:
import heapq

stopwords = nltk.corpus.stopwords.words('russian')

def freq_summarize(source_text, num_sentences=3):
    text = Text(source_text)
    
    word_frequencies = {}
    
    for sentence in text.sentences:
        for word in sentence.clean_tokens:
            if word.token not in stopwords:
                if word not in word_frequencies.keys():
                    word_frequencies[word.token] = 1
                else:
                    word_frequencies[word.token] += 1
    
    maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
        
    sentence_scores = {}
    for sentence in text.sentences:
        for word in sentence.clean_tokens:
            if word.token in word_frequencies.keys():
                if sentence.sentence not in sentence_scores.keys():
                    sentence_scores[sentence.sentence] = word_frequencies[word.token]
                else:
                    sentence_scores[sentence.sentence] += word_frequencies[word.token]
                    
    summary_sentences = heapq.nlargest(num_sentences, sentence_scores, key=sentence_scores.get)
    
    summary = ''
    for sentence in text.sentences:
        if sentence.sentence in summary_sentences:
            summary += sentence.sentence + ' '

    return summary

In [75]:
print(freq_summarize(source_txt))

И вот этот Сардаров — делал интервью с Собяниным (дифирамбы пел), а также задавал вопросы на прямой линии с Путиным. Эта парочка Сардаров — Эдвард Бил, давно вызывает вопросы в блогерском сообществе. Поэтому ответ на вопрос «а как так вышло?», напрашивается довольно очевидный. 


# Поиск упоминаний

In [76]:
from russiannames.parser import NamesParser

In [77]:
def normalize_name(name):
    def get_noun(n):
        name_forms = pymorphy.parse(n)
        normal_form = list(filter(lambda x: 'Name' in x.tag or 'Surn' in x.tag, name_forms))
        
        normal_form = normal_form[0].normal_form if len(normal_form) > 0 else n
        
        return normal_form if len(normal_form) <= len(n) else n

    norm_name = ''
    for name_part in name.split(' '):
        norm_name += get_noun(name_part) + ' '
        
    return norm_name[:-1].lower()

def count_persons(text_str):
    parser = NamesParser()
    markup = ner(text_str)
    
    person_spans = list(filter(lambda x: x.type == 'PER', markup.spans))

    names = {}

    def put_to_names(name):
        if name in names:
            names[name] += 1
            return True

        if name[:-1] in names:
            names[name[:-1]] += 1
            return True

        return False

    for span in person_spans:
        put = False

        span_text = text_str[span.start:span.stop]
        name = parser.parse(span_text)

        if not name['text']:
            continue

        name = normalize_name(name['text'])

        name_parts = name.split(' ')        

        if len(name_parts) == 1:
            if put_to_names(name):
                continue

        names_keys = names.keys()

        for names_key in names_keys:
            if len(list(set(name_parts) & set(names_key.split(' ')))) == len(name_parts):
                if put_to_names(names_key):
                    put = True
                    break

        if not put:
            names[name] = 1
            
    return names

In [78]:
count_persons(text.text)

{'эдвард': 1, 'амиран сардаров': 3, 'собянин': 1, 'путин': 1, 'эдвард бил': 1}

In [79]:
def identify_persons(text, *persons_to_check):
    names = count_persons(text)
    identified = {}

    for p in persons_to_check:
        person = p.lower()
        if names.get(person):
            identified[p] = names[person]
            
    return identified

In [80]:
identify_persons(text.text, 'Сергей ломакин')

{}

# Выделение контактной информации

In [81]:
import re


emails = re.compile("[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+")
urls = re.compile('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+')
phones = re.compile("((?:\+\d{2}[-\.\s]??|\d{4}[-\.\s]??)?(?:\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}))")

In [82]:
text = """
email: s.chernobrovkin@ktsstudio.ru
phone: 8(911)123-1232
url: https://kts.studio
"""

In [83]:
phones.findall(text)

['(911)123-1232']

In [84]:
emails.findall(text)

['s.chernobrovkin@ktsstudio.ru']

In [85]:
urls.findall(text)

['https://kts.studio']

# Сентимент-анализ

In [36]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

import stanza

In [37]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [38]:
post_txt = """
⚡️Навальный объявил голодовку.

"Почему заключённые объявляют голодовки? А нету у тебя других методов борьбы, вот и объявляешь", - объяснил Алексей у себя в Инстаграме.
"""

comments = [
    "- 1 враг народа.  Не зря Сталин стрелял за голодовку или срок прибавлял.",
    "Мне кажется им будет плевать",
    "В Беларуси голодают и Луке насрать)",
    "Хотелось бы чтобы общество осознало, что Навальный это просто полит заключённый. А в какой-то степени символ борьбы с режимом"
]

results = model.predict(comments, k=2)

In [39]:
for message, sentiment in zip(comments, results):
    print(message, '->', sentiment)

- 1 враг народа.  Не зря Сталин стрелял за голодовку или срок прибавлял. -> {'negative': 0.4765896201133728, 'skip': 0.4688006341457367}
Мне кажется им будет плевать -> {'negative': 0.4532718360424042, 'neutral': 0.3629792034626007}
В Беларуси голодают и Луке насрать) -> {'neutral': 0.348655104637146, 'skip': 0.20690405368804932}
Хотелось бы чтобы общество осознало, что Навальный это просто полит заключённый. А в какой-то степени символ борьбы с режимом -> {'negative': 0.6370407938957214, 'neutral': 0.275139719247818}


# Полярность текста

In [40]:
from polyglot.downloader import downloader
from polyglot.text import Text

In [75]:
text = Text(source_txt)

In [76]:
text.polarity

1.0